In [ ]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
import os

# Charger les données
df = pd.read_parquet("data/processed/games_raw.parquet")

# Filtrage des ouvertures inutiles
df = df[df["main_opening"].notna() &
        (df["main_opening"] != "?") &
        (df["main_opening"] != "King's Pawn") &
        (df["main_opening"] != "Queen's Pawn")]

# Blancs
white_openings = df.groupby("main_opening").agg(
    games_white=("white_score", "count"),
    winrate_white=("white_score", "mean")
).reset_index()
white_openings = white_openings[white_openings["games_white"] >= 30]
top_white_winrate = white_openings.sort_values("winrate_white", ascending=False).head(10)
top_white_games = white_openings.sort_values("games_white", ascending=False).head(10)

# Noirs
black_openings = df.groupby("main_opening").agg(
    games_black=("black_score", "count"),
    winrate_black=("black_score", "mean")
).reset_index()
black_openings = black_openings[black_openings["games_black"] >= 30]
top_black_winrate = black_openings.sort_values("winrate_black", ascending=False).head(10)
top_black_games = black_openings.sort_values("games_black", ascending=False).head(10)

# ----------- GRAPHIQUE BLANCS ----------- #
fig_white = go.Figure()

# Trace pour tri par Winrate
fig_white.add_trace(go.Bar(
    x=top_white_winrate["main_opening"],
    y=top_white_winrate["winrate_white"],
    customdata=top_white_winrate[["games_white"]],
    marker=dict(
        color=top_white_winrate["winrate_white"],
        colorscale=["#489fde", "#1f68ab", "#154d8c", "#042749", "#001029"],
        line=dict(color='black', width=1.5)
    ),
    text=top_white_winrate["winrate_white"].apply(lambda x: f"{x:.2%}"),
    textposition="outside",
    name="Trié par Winrate",
    hovertemplate="<b>%{x}</b><br>Winrate: %{y:.2%}<br>Parties: %{customdata[0]}"
))

# Trace pour tri par Nombre de parties
fig_white.add_trace(go.Bar(
    x=top_white_games["main_opening"],
    y=top_white_games["games_white"],
    marker=dict(
        color=top_white_games["games_white"],
        colorscale=["#cce5ff", "#3399ff", "#0055cc", "#003d99", "#001f4d"],
        line=dict(color='black', width=1.5)
    ),
    text=top_white_games["games_white"],
    textposition="outside",
    name="Trié par Nombre de parties",
    hovertemplate="<b>%{x}</b><br>Parties: %{y}"
))

# Layout Blancs
fig_white.update_layout(
    updatemenus=[dict(
        type="buttons",
        direction="right",
        x=0.57, y=1.15,
        buttons=list([
            dict(label="Winrate",
                 method="update",
                 args=[{"visible": [True, False]},
                       {"title": "♙ <b>Top 10 Ouvertures Blancs</b> (par Winrate)"}]),
            dict(label="Nombre de parties",
                 method="update",
                 args=[{"visible": [False, True]},
                       {"title": "♙ <b>Top 10 Ouvertures Blancs</b> (par Nombre de parties)"}])
        ])
    )],
    title="♙ <b>Top 10 Ouvertures Blancs</b> (par Winrate)",
    xaxis_title="Ouverture",
    yaxis_title="Winrate / Nombre de parties",
    xaxis_tickangle=-45,
    transition={'duration': 500, 'easing': 'cubic-in-out'},
    hoverlabel=dict(bgcolor="white", font_size=14, font_family="Rockwell")
)

# Annotation meilleure ouverture
best_white = top_white_winrate.iloc[0]
fig_white.add_annotation(
    x=best_white["main_opening"],
    y=best_white["winrate_white"],
    showarrow=True,
    arrowhead=2,
    ax=0,
    ay=-40
)

# ----------- GRAPHIQUE NOIRS ----------- #
fig_black = go.Figure()

fig_black.add_trace(go.Bar(
    x=top_black_winrate["main_opening"],
    y=top_black_winrate["winrate_black"],
    customdata=top_black_winrate[["games_black"]],
    marker=dict(
        color=top_black_winrate["winrate_black"],
        colorscale=["#e85151", "#C22E2E", "#a30e0e", "#720000", "#4d0a0a"],
        line=dict(color='black', width=1.5)
    ),
    text=top_black_winrate["winrate_black"].apply(lambda x: f"{x:.2%}"),
    textposition="outside",
    name="Trié par Winrate",
    hovertemplate="<b>%{x}</b><br>Winrate: %{y:.2%}<br>Parties: %{customdata[0]}"
))

fig_black.add_trace(go.Bar(
    x=top_black_games["main_opening"],
    y=top_black_games["games_black"],
    marker=dict(
        color=top_black_games["games_black"],
        colorscale=["#ffd6d6", "#ff9999", "#cc3333", "#800000", "#4d0000"],
        line=dict(color='black', width=1.5)
    ),
    text=top_black_games["games_black"],
    textposition="outside",
    name="Trié par Nombre de parties",
    hovertemplate="<b>%{x}</b><br>Parties: %{y}"
))

fig_black.update_layout(
    updatemenus=[dict(
        type="buttons",
        direction="right",
        x=0.57, y=1.15,
        buttons=list([
            dict(label="Winrate",
                 method="update",
                 args=[{"visible": [True, False]},
                       {"title": "♟️ <b>Top 10 Ouvertures Noirs</b> (par Winrate)"}]),
            dict(label="Nombre de parties",
                 method="update",
                 args=[{"visible": [False, True]},
                       {"title": "♟️ <b>Top 10 Ouvertures Noirs</b> (par Nombre de parties)"}])
        ])
    )],
    title="♟️ <b>Top 10 Ouvertures Noirs</b> (par Winrate)",
    xaxis_title="Ouverture",
    yaxis_title="Winrate / Nombre de parties",
    xaxis_tickangle=-45,
    transition={'duration': 500, 'easing': 'cubic-in-out'},
    hoverlabel=dict(bgcolor="white", font_size=14, font_family="Rockwell")
)

# Annotation meilleure ouverture
best_black = top_black_winrate.iloc[0]
fig_black.add_annotation(
    x=best_black["main_opening"],
    y=best_black["winrate_black"],
    showarrow=True,
    arrowhead=2,
    ax=0,
    ay=-40
)

# ----------- Sauvegarde HTML ----------- #
os.makedirs("data/exported", exist_ok=True)
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
fig_white.write_html(f"data/exported/top_openings_white_{timestamp}.html")
fig_black.write_html(f"data/exported/top_openings_black_{timestamp}.html")

# ----------- Affichage ----------- #
fig_white.show()
fig_black.show()
